In [25]:
import os
from openai import OpenAI

In [26]:
client=OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [27]:
completion=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"user",
        "content":"Mention some of the areas you are not trained on"
        }
        ]
    )
print(completion)

ChatCompletion(id='chatcmpl-8xJhmdeqobAmKl7kDRyu1EctY7HEI', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. Medical procedures\n2. Legal advice\n3. Financial planning\n4. Engineering design\n5. Psychology and counseling techniques', role='assistant', function_call=None, tool_calls=None))], created=1709147790, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_86156a94a0', usage=CompletionUsage(completion_tokens=26, prompt_tokens=18, total_tokens=44))


In [28]:
print(completion.usage)

CompletionUsage(completion_tokens=26, prompt_tokens=18, total_tokens=44)


In [29]:
print(completion.choices[0].message.content)

1. Medical procedures
2. Legal advice
3. Financial planning
4. Engineering design
5. Psychology and counseling techniques


In [30]:
from langchain.document_loaders import DirectoryLoader

In [31]:
pwd

'D:\\Projects\\MQA\\notebooks'

In [32]:
os.listdir('..//data')

['DSMP 2.0 FAQs.pdf']

In [33]:
directory='..//data'

In [34]:
documents=DirectoryLoader(directory).load()

D:\Projects\MQA\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
len(documents)

1

In [36]:
documents[0].metadata

{'source': '..\\data\\DSMP 2.0 FAQs.pdf'}

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [38]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs=text_splitter.split_documents(documents)

In [39]:
len(docs)

4

In [40]:
docs[0].page_content

'1. About the Program\n\na. What is the course fee for Data Science Mentorship Program (DSMP\n\n2.0)\n\nThe total course fee for the course is Rs 10,600.\n\nb. What is the total duration of the course?\n\nThe total duration of the course is 6-8 months.\n\nc. Will Deep Learning and NLP be a part of this program?\n\nNo, NLP and Deep Learning both are not a part of this program’s\n\ncurriculum.\n\nd. What if I miss a live session? Will I get a recording of the session?\n\nYes all our sessions are recorded, so even if you miss a session you can\n\ngo back and watch the recording.\n\ne. Where can I find the class schedule?\n\nYou will find the class schedule in your course dashboard once you\n\nenroll for the course.\n\nf. What is the time duration of all the live sessions?\n\nRoughly, all the sessions last 2 hours.\n\ng. What is the language spoken by the instructor during the sessions?\n\nHinglish\n\nh. How will I be informed about the upcoming class?\n\nYou will get a mail from our side 

In [41]:
import pandas as pd

In [42]:
df=pd.DataFrame(docs)

In [43]:
df.head()

,0,1,2
0,"(page_content, 1. About the Program\n\na. What...","(metadata, {'source': '..\data\DSMP 2.0 FAQs.p...","(type, Document)"
1,"(page_content, become a paid user.\n\ni. Can I...","(metadata, {'source': '..\data\DSMP 2.0 FAQs.p...","(type, Document)"
2,"(page_content, b. What if I don’t like the cou...","(metadata, {'source': '..\data\DSMP 2.0 FAQs.p...","(type, Document)"
3,"(page_content, You will get doubt support for ...","(metadata, {'source': '..\data\DSMP 2.0 FAQs.p...","(type, Document)"


In [20]:
df.columns

RangeIndex(start=0, stop=3, step=1)

In [44]:
df.rename(columns={0:"Content",1:"Metadata",2:"Type"},inplace=True)

In [22]:
df.head(2)

,Content,Metadata,Type
0,"(page_content, 1. About the Program\n\na. What...","(metadata, {'source': '..\data\DSMP 2.0 FAQs.p...","(type, Document)"
1,"(page_content, become a paid user.\n\ni. Can I...","(metadata, {'source': '..\data\DSMP 2.0 FAQs.p...","(type, Document)"


In [45]:
df['Content']=df['Content'].apply(lambda x: x[1])

In [46]:
df.head(2)

,Content,Metadata,Type
0,1. About the Program\n\na. What is the course ...,"(metadata, {'source': '..\data\DSMP 2.0 FAQs.p...","(type, Document)"
1,become a paid user.\n\ni. Can I do this course...,"(metadata, {'source': '..\data\DSMP 2.0 FAQs.p...","(type, Document)"


In [47]:
df['Content']=df['Content'].str.replace("\n\n"," ")

In [48]:
df.drop(['Type','Metadata'],axis=1,inplace=True)

In [49]:
df.head(2)

,Content
0,1. About the Program a. What is the course fee...
1,become a paid user. i. Can I do this course if...


In [28]:
df['Len']=df['Content'].apply(len)

In [30]:
df.head()

,Content,Len
0,1. About the Program a. What is the course fee...,973
1,become a paid user. i. Can I do this course if...,948
2,b. What if I don’t like the course after makin...,955
3,You will get doubt support for 1 year from the...,976


In [32]:
import tiktoken

In [33]:
enc=tiktoken.get_encoding("cl100k_base")
tokens=enc.encode("My Name is Rajeev Ranjan")
tokens

[5159, 4076, 374, 26291, 2176, 85, 39431, 23685]

In [34]:
df['Tokens']=df['Content'].apply(lambda x: len(enc.encode(x)))

In [35]:
df.head()

,Content,Len,Tokens
0,1. About the Program a. What is the course fee...,973,233
1,become a paid user. i. Can I do this course if...,948,229
2,b. What if I don’t like the course after makin...,955,224
3,You will get doubt support for 1 year from the...,976,214


In [36]:
df['Tokens'].sum()

900

In [37]:
df['Tokens'].mean()

225.0

In [38]:
df['words']=df['Content'].apply(lambda x: len(x.split()))

In [39]:
df.head(2)

,Content,Len,Tokens,words
0,1. About the Program a. What is the course fee...,973,233,187
1,become a paid user. i. Can I do this course if...,948,229,177


In [40]:
pricing=0.00002*df['Tokens'].sum()/1000*83
pricing

0.001494

In [42]:
#pd.set_option('display.max_colwidth',None)

In [43]:
embedding=client.embeddings.create(input=[df['Content'][0]],model='text-embedding-3-small')

In [44]:
embedding

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.005896944552659988, 0.006915039382874966, 0.05431365966796875, -0.028438027948141098, -0.00893407128751278, 0.022134989500045776, 0.02695092372596264, 0.020247509703040123, -0.001642965362407267, 0.016941560432314873, 0.06099419295787811, -0.04735858365893364, -0.05394760146737099, -0.033768728375434875, 0.005159111227840185, 0.027111072093248367, -0.04392680153250694, 0.030085284262895584, -0.045322392135858536, 0.04193636775016785, 0.009225772693753242, 0.04896008223295212, -0.02740849368274212, -0.0014299089089035988, 0.02896423451602459, -0.011873963288962841, -0.005819729063659906, 0.009357323870062828, 0.028643935918807983, 0.01747920736670494, 0.02425125427544117, 0.0012869180645793676, -0.02420549839735031, -0.008527976460754871, -0.01603785902261734, -0.015225670300424099, 0.0008329219417646527, 0.04735858365893364, -0.018863357603549957, -0.011885402724146843, -0.01630096137523651, -0.02528078854084015, 0.00083220697706565

In [47]:
embedding.data[0].embedding

[0.005145154427736998,
 -0.003077789442613721,
 0.00574210612103343,
 0.006207263097167015,
 0.011277475394308567,
 0.007416671607643366,
 -0.022182825952768326,
 0.05743139609694481,
 -0.038949154317379,
 -0.03342929109930992,
 0.055405378341674805,
 -0.07827043533325195,
 -0.006186589598655701,
 0.007633745204657316,
 0.05486786365509033,
 -0.05796891078352928,
 -0.005186501890420914,
 -0.04506855458021164,
 -0.036364950239658356,
 0.027537301182746887,
 0.04606088995933533,
 0.03070036880671978,
 -0.010021551512181759,
 0.0033207049127668142,
 0.028136836364865303,
 0.029232539236545563,
 -0.01540186908096075,
 -0.03475240617990494,
 0.0403963103890419,
 0.005067628342658281,
 0.006936009507626295,
 -0.009628752246499062,
 0.00992851983755827,
 -0.06553547084331512,
 0.03233358636498451,
 0.03146529570221901,
 -0.02139722742140293,
 -0.04403487220406532,
 0.022803034633398056,
 0.002204327844083309,
 -0.0013941791839897633,
 -0.03204415738582611,
 0.003842714475467801,
 0.0523870289

In [45]:
len(embedding.data[0].embedding)

1536

In [50]:
def get_embedding(text,model='text-embedding-3-small'):
    embedding=client.embeddings.create(input=[text],model=model)
    return embedding.data[0].embedding

In [70]:
# def get_embedding(text, model="text-embedding-3-small"):
#    return client.embeddings.create(input = text, model=model).data[0].embedding

In [47]:
get_embedding("My name is Rajeev Ranjan. I live in Bangalore. My Native Place is Bihar.")

[-0.009290183894336224,
 -0.009793568402528763,
 0.030342616140842438,
 0.020434416830539703,
 -0.0435003861784935,
 -0.014463580213487148,
 -0.009778616018593311,
 0.0171051025390625,
 -0.03951318562030792,
 -0.014403771609067917,
 -0.00018409664335194975,
 -0.006145277060568333,
 -0.01954726316034794,
 -0.006843037437647581,
 0.03534655645489693,
 -0.008218622766435146,
 0.01865014247596264,
 0.00601070886477828,
 -0.02246789075434208,
 0.030960632488131523,
 0.03239602595567703,
 -0.00837312638759613,
 0.03147897124290466,
 -0.014144603163003922,
 0.044656675308942795,
 0.0459325797855854,
 -0.010516248643398285,
 -0.0033492506481707096,
 0.029644856229424477,
 -0.043021924793720245,
 0.01586906984448433,
 -0.013287355192005634,
 -0.034289948642253876,
 -0.017254622653126717,
 0.00019063815125264227,
 -0.029325880110263824,
 0.018131807446479797,
 -0.062080755829811096,
 -0.025278868153691292,
 0.022049233317375183,
 -0.028309142217040062,
 -0.01564977318048477,
 0.03102044202387333

In [51]:
df['Embedding']=df.Content.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))

In [52]:
df.head(2)

,Content,Embedding
0,1. About the Program a. What is the course fee...,"[-0.005896944552659988, 0.006915039382874966, ..."
1,become a paid user. i. Can I do this course if...,"[0.009789426811039448, 0.02742091938853264, 0...."


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Content    4 non-null      object
 1   Embedding  4 non-null      object
dtypes: object(2)
memory usage: 196.0+ bytes


In [54]:
temp=pd.read_csv("..\\Embeddings\\embedding.csv")

In [55]:
temp.head(2)

,Content,Embedding
0,1. About the Program a. What is the course fee...,"[-0.005896944552659988, 0.006915039382874966, ..."
1,become a paid user. i. Can I do this course if...,"[0.009789426811039448, 0.02742091938853264, 0...."


In [56]:
type(df['Embedding'][0])

list

In [57]:
type(temp['Embedding'][0])

str

In [61]:
from ast import literal_eval

In [67]:
temp["Embedding"]=temp['Embedding'].apply(literal_eval)

In [68]:
temp['Embedding'][0]

[-0.005896944552659988,
 0.006915039382874966,
 0.05431365966796875,
 -0.028438027948141098,
 -0.00893407128751278,
 0.022134989500045776,
 0.02695092372596264,
 0.020247509703040123,
 -0.001642965362407267,
 0.016941560432314873,
 0.06099419295787811,
 -0.04735858365893364,
 -0.05394760146737099,
 -0.033768728375434875,
 0.005159111227840185,
 0.027111072093248367,
 -0.04392680153250694,
 0.030085284262895584,
 -0.045322392135858536,
 0.04193636775016785,
 0.009225772693753242,
 0.04896008223295212,
 -0.02740849368274212,
 -0.0014299089089035988,
 0.02896423451602459,
 -0.011873963288962841,
 -0.005819729063659906,
 0.009357323870062828,
 0.028643935918807983,
 0.01747920736670494,
 0.02425125427544117,
 0.0012869180645793676,
 -0.02420549839735031,
 -0.008527976460754871,
 -0.01603785902261734,
 -0.015225670300424099,
 0.0008329219417646527,
 0.04735858365893364,
 -0.018863357603549957,
 -0.011885402724146843,
 -0.01630096137523651,
 -0.02528078854084015,
 0.0008322069770656526,
 0.0

In [51]:
# df.nlargest(1,'Tokens')

In [52]:
df

,Content,Len,Tokens,words,Embedding
0,1. About the Program a. What is the course fee...,973,233,187,"[-0.005896944552659988, 0.006915039382874966, ..."
1,become a paid user. i. Can I do this course if...,948,229,177,"[0.009789426811039448, 0.02742091938853264, 0...."
2,b. What if I don’t like the course after makin...,955,224,189,"[-0.0035533132031559944, 0.03466103971004486, ..."
3,You will get doubt support for 1 year from the...,976,214,174,"[0.03474213182926178, 0.004566950257867575, 0...."


In [53]:
df.to_csv("Embeddings.csv")

In [56]:
df['Content'][3]

'You will get doubt support for 1 year from the data of registration.. f. I am living outside India and I am not able to make the payment on the website, what should I do? You have to contact us by sending a mail at support@campusx.in 4. Certificate and Placement Assistance related queries a. What is the criteria to get the certificate? There are 2 criterias: 1. You have to pay the entire fee of Rs 10,600 2. You have to attempt all the course assessments. b. I have read that Job Preparation is a part of this program. What comes under Job Preparation? This is to clarify that Job Preparation does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under Job Preparation 1. Portfolio Building sessions 2. Interview Questions 3. Sessions with industry mentors 4. Discussion on Job hunting strategies'

In [66]:
q="What to do when I am outside india"

In [67]:
content=f'''Answer the question from below context only with 100% certainity. Otherwise answer: "Could not find answer in the context"
Context: "I am living outside India and I am not able to make the payment on the website, what should I do? You have to contact us by sending a mail at support@campusx.in 4. Certificate and Placement Assistance related queries a. What is the criteria to get the certificate? There are 2 criterias: 1. You have to pay the entire fee of Rs 10,600 2. You have to attempt all the course assessments."
Q: {q}
A: 
'''

In [68]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role":"user","content":content}
    ]
)

In [69]:
response.choices[0].message.content

'You have to contact us by sending a mail at support@campusx.in'

In [70]:
df

,Content,Len,Tokens,words,Embedding
0,1. About the Program a. What is the course fee...,973,233,187,"[-0.005896944552659988, 0.006915039382874966, ..."
1,become a paid user. i. Can I do this course if...,948,229,177,"[0.009789426811039448, 0.02742091938853264, 0...."
2,b. What if I don’t like the course after makin...,955,224,189,"[-0.0035533132031559944, 0.03466103971004486, ..."
3,You will get doubt support for 1 year from the...,976,214,174,"[0.03474213182926178, 0.004566950257867575, 0...."


In [72]:
qve=client.embeddings.create(
    model="text-embedding-3-small",
    input=q
)
qve.data[0].embedding

[-0.0008144689491018653,
 -0.009970411658287048,
 0.03305978700518608,
 0.04664520174264908,
 -0.059880778193473816,
 -0.027418632060289383,
 0.022331390529870987,
 -0.014729679562151432,
 0.018614349886775017,
 0.04145592078566551,
 0.03746192529797554,
 -0.015859369188547134,
 -0.020727960392832756,
 -0.02227308414876461,
 0.025596553459763527,
 0.021413061767816544,
 -0.002560019725933671,
 0.02147137001156807,
 -0.025654859840869904,
 0.090083546936512,
 0.0056192888878285885,
 0.06326255202293396,
 0.01750652678310871,
 0.023380907252430916,
 0.015669872984290123,
 -0.01730245351791382,
 -0.011697742156684399,
 0.01564071886241436,
 0.006916608661413193,
 -0.00018232170259580016,
 0.008826146833598614,
 -0.03644156455993652,
 -0.011493668891489506,
 -0.040668785572052,
 0.007944260723888874,
 0.030348533764481544,
 -0.012477591633796692,
 -0.023716170340776443,
 0.019299451261758804,
 0.006326255388557911,
 0.023278871551156044,
 -0.05043512210249901,
 0.047840483486652374,
 0.019

In [73]:
qve=qve.data[0].embedding

In [79]:
vec=[]
for i in range(df.shape[0]):
    vec.append(np.dot(qve,df['Embedding'][i]))
print(vec,max(vec))

[0.23314115236500682, 0.3043523231526389, 0.317278923416454, 0.343761957640194] 0.343761957640194


In [81]:
vec_np=np.array(vec)

In [82]:
vec_np

array([0.23314115, 0.30435232, 0.31727892, 0.34376196])

In [83]:
max_index=np.argmax(vec_np)
max_index

3

In [85]:
ctxt=df['Content'][max_index]
ctxt

'You will get doubt support for 1 year from the data of registration.. f. I am living outside India and I am not able to make the payment on the website, what should I do? You have to contact us by sending a mail at support@campusx.in 4. Certificate and Placement Assistance related queries a. What is the criteria to get the certificate? There are 2 criterias: 1. You have to pay the entire fee of Rs 10,600 2. You have to attempt all the course assessments. b. I have read that Job Preparation is a part of this program. What comes under Job Preparation? This is to clarify that Job Preparation does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under Job Preparation 1. Portfolio Building sessions 2. Interview Questions 3. Sessions with industry mentors 4. Discussion on Job hunting strategies'

In [105]:
question=input("What is your query? ")
def get_ques_embedding(ques):
    return client.embeddings.create(input=ques,model='text-embedding-3-small').data[0].embedding
qe=get_ques_embedding(question)   

What is your query?  is NLP and deep learning covered in this course


In [106]:
def get_context(query_embedding,dataframe):
    cosine_similarity=[]
    for i in range(dataframe.shape[0]):
        cosine_similarity.append(np.dot(query_embedding,dataframe['Embedding'][0]))
    index = np.argmax(np.array(cosine_similarity))
    return dataframe['Content'][index]
    

In [107]:
context=get_context(qe,df)

In [108]:
content=f'''Answer the question from below context only with 80% certainity. Otherwise answer: "Could not find answer in the context"
Context: {context}
Q: {question}
A: 
'''

In [109]:
content

'Answer the question from below context only with 80% certainity. Otherwise answer: "Could not find answer in the context"\nContext: 1. About the Program a. What is the course fee for Data Science Mentorship Program (DSMP 2.0) The total course fee for the course is Rs 10,600. b. What is the total duration of the course? The total duration of the course is 6-8 months. c. Will Deep Learning and NLP be a part of this program? No, NLP and Deep Learning both are not a part of this program’s curriculum. d. What if I miss a live session? Will I get a recording of the session? Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording. e. Where can I find the class schedule? You will find the class schedule in your course dashboard once you enroll for the course. f. What is the time duration of all the live sessions? Roughly, all the sessions last 2 hours. g. What is the language spoken by the instructor during the sessions? Hinglish h. How will I 

In [110]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role":"user","content":content}
    ]
)

In [111]:
response.choices[0].message.content

'No, NLP and Deep Learning both are not a part of this program’s curriculum.'

In [122]:
## Finalising the code
question=input("What is your query? ")
def get_ques_embedding(ques):
    return client.embeddings.create(input=ques,model='text-embedding-3-small').data[0].embedding
    
qe=get_ques_embedding(question) 

def get_context(query_embedding,dataframe):
    cosine_similarity=[]
    for i in range(dataframe.shape[0]):
        cosine_similarity.append(np.dot(query_embedding,dataframe['Embedding'][i]))
    index = np.argmax(np.array(cosine_similarity))
    #print("MATCHING INDEX: ",index)
    return dataframe['Content'][index]
    
    
context=get_context(qe,df)

content=f'''Answer the question from below context only with 80% certainity. Otherwise answer: "Could not find answer in the context"
Context: {context}
Q: {question}
A: 
'''
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role":"user","content":content}
    ]
)
print(response.choices[0].message.content)

What is your query?  how to do the payment?


You have to make all your monthly payments on the website provided - https://learnwith.campusx.in/


In [2]:
print("Hi")

Hi


In [3]:
pwd

'D:\\Projects\\MQA\\notebooks'

In [7]:
os.listdir("..\\Embeddings")

['embedding.csv']

In [9]:
import pandas as pd

In [21]:
temp=pd.read_csv("..\\Embeddings\\embedding.csv")

In [22]:
temp.head(2)

,Content,Embedding
0,1. About the Program a. What is the course fee...,"[-0.005896944552659988, 0.006915039382874966, ..."
1,become a paid user. i. Can I do this course if...,"[0.009789426811039448, 0.02742091938853264, 0...."


In [23]:
temp.shape

(4, 2)

In [24]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Content    4 non-null      object
 1   Embedding  4 non-null      object
dtypes: object(2)
memory usage: 196.0+ bytes


In [13]:
temp.to_csv("one.csv")

In [14]:
temp1=pd.read_csv("one.csv")

In [15]:
temp1.head(2)

,Unnamed: 0.1,Unnamed: 0,Content,Embedding
0,0,0,1. About the Program a. What is the course fee...,"[-0.005896944552659988, 0.006915039382874966, ..."
1,1,1,become a paid user. i. Can I do this course if...,"[0.009789426811039448, 0.02742091938853264, 0...."


In [19]:
temp1.drop(columns=["Unnamed: 0.1","Unnamed: 0"]).to_csv("two.csv",index=False)

In [20]:
pd.read_csv("two.csv")

,Content,Embedding
0,1. About the Program a. What is the course fee...,"[-0.005896944552659988, 0.006915039382874966, ..."
1,become a paid user. i. Can I do this course if...,"[0.009789426811039448, 0.02742091938853264, 0...."
2,b. What if I don’t like the course after makin...,"[-0.0035533132031559944, 0.03466103971004486, ..."
3,You will get doubt support for 1 year from the...,"[0.03474213182926178, 0.004566950257867575, 0...."
